In [9]:
from collections import defaultdict

In [19]:
with open("22.in") as f:
    d = f.read()
    lines = d.split("\n")
nums = [int(l) for l in lines]

In [2]:
def next_secret(secret):
    m = 16777216
    secret = (secret ^ (secret << 6)) % m
    secret = (secret ^ (secret >> 5)) % m
    secret = (secret ^ (secret << 11)) % m
    
    return secret

In [6]:
t = 0
for n in nums:
    for i in range(2000):
        n = next_secret(n)
    t += n
t

14180628689

In [7]:
len(lines)

1729

# Part 2

In [27]:
won = defaultdict(int)
all_price = []
monkey_had = 0
for n in nums:
    changes = tuple()
    won_this_monkey = {}
    for i in range(2000):
        next_n = next_secret(n)
        change = next_n % 10 - n % 10
        if len(changes) < 4:
            changes += (change,)
        else:
            changes = changes[1:] + (change,)
            
        n = next_n
            
        if len(changes) == 4:
            all_price.append(next_n%10)
            if changes in won_this_monkey:
                continue
            if changes == (0, 0, -1, 2):
                #print(next_n % 10)
                monkey_had += 1
            won_this_monkey[changes] = next_n % 10
    for k,v in won_this_monkey.items():
        won[k] += v

In [21]:
max(won.values())

1690

In [72]:
len(nums)

1729

In [67]:
4.5*len(nums)*1997/10000

1553.76585

In [25]:
[k for k,v in won.items() if v == 1690]

[(0, 0, -1, 2)]

In [70]:
won[(0, 0, -1, 2)]

1690

In [75]:
import numpy as np

In [76]:
np.sqrt(1729*0.15976*(1 - 0.15976))

15.234675172434757

In [71]:
8/10**5 * 1997

0.15976

In [69]:
won[(0,0,0,2)]

1306

In [16]:
won[(-2,1,-1,3)]

16

In [23]:
sum(all_price)/len(all_price)

4.501288659420594

In [24]:
len(nums)

1729

In [28]:
monkey_had

311

In [29]:
1690 / 311

5.434083601286174

# Beyond

In [30]:
won = defaultdict(int)
all_price = []
monkey_had = 0
n_9 = 0
for n in nums:
    changes = tuple()
    won_this_monkey = {}
    has_9 = False
    for i in range(2000):
        next_n = next_secret(n)
        has_9 = True
        change = next_n % 10 - n % 10
        if len(changes) < 4:
            changes += (change,)
        else:
            changes = changes[1:] + (change,)
            
        n = next_n
            
        if len(changes) == 4:
            all_price.append(next_n%10)
            if changes in won_this_monkey:
                continue
            if changes == (0, 0, -1, 2):
                #print(next_n % 10)
                monkey_had += 1
            won_this_monkey[changes] = next_n % 10
    if has_9:
        n_9 += 1
    for k,v in won_this_monkey.items():
        won[k] += v

In [32]:
n_9

1729

In [33]:
len(lines)

1729

In [36]:
best_threshold = [0]
expected = [4.5]

while(len(best_threshold) < 2001):
    bt = None
    e = None
    for test_threshold in range(10):
        value = expected[-1]*test_threshold/10  + (45 - (test_threshold - 1)*test_threshold/2)/10
        if bt is None or value > e:
            e = value
            bt = test_threshold
    best_threshold.append(bt)
    expected.append(e)

In [40]:
best_threshold[:11]

[0, 5, 6, 7, 7, 8, 8, 8, 8, 8, 9]

In [41]:
expected[:10]

[4.5,
 5.75,
 6.45,
 6.914999999999999,
 7.240499999999999,
 7.492399999999999,
 7.693919999999999,
 7.855136,
 7.9841088000000004,
 8.08728704]

In [51]:
num_evolution = [nums]

current_nums = nums
for _ in range(9):
    current_nums = [next_secret(n) for n in current_nums]
    num_evolution.append(current_nums)
    
price_evolution = []
for gen in num_evolution:
    price_evolution.append([n % 10 for n in gen])

In [52]:
remaining_value = [[0 for i in range(len(nums))] for t in range(9)]
remaining_value.append(price_evolution[-1])


for t in range(8,-1,-1):
    for i in range(len(nums)):
        remaining_value[t][i] = max(remaining_value[t+1][i],price_evolution[t][i])
        

In [56]:
good_policy = [0, 5, 6, 7, 7, 8, 8, 8, 8, 8][::-1]
indices = range(len(nums))
value = 0
for t in range(10):
    threshold = good_policy[t]
    pe = price_evolution[t]
    new_indices = []
    for i in indices:
        if pe[i] >= threshold:
            value += pe[i]
        else:
            new_indices.append(i)
            
    indices = new_indices

In [57]:
value/len(nums)

8.068825910931174

In [61]:
best = value
def optimal(value_so_far,t,indices):
    global best
    if t == 9:
        pe = price_evolution[t]
        for i in indices:
            value_so_far += price_evolution[t][i]
        if value_so_far > best:
            best = value_so_far
        return value_so_far,[0]
    
    my_best_value = None
    my_best_policy = None
    
    for test_threshold in range(10):
        new_indices = []
        local_value_so_far = value_so_far
        pe = price_evolution[t]
        for i in indices:
            if pe[i] >= test_threshold:
                local_value_so_far += pe[i]
            else:
                new_indices.append(i)
        max_possible = local_value_so_far
        rv = remaining_value[t+1]
        for i in new_indices:
            max_possible += rv[i]
            
        if max_possible < best:
            continue
            
        value, policy = optimal(local_value_so_far,t+1,new_indices)
        
        if value is None:
            continue
            
        if my_best_value is None or value > my_best_value:
            my_best_value = value
            my_best_policy = [test_threshold] + policy
            
            
    return my_best_value, my_best_policy
            
                
                

In [62]:
optimal(0,0,list(range(len(nums))))

(13955, [8, 8, 8, 8, 8, 7, 7, 6, 4, 0])

In [63]:
value

13951

In [64]:
good_policy

[8, 8, 8, 8, 8, 7, 7, 6, 5, 0]